In [ ]:
#Importando os módulos necessários. Para calcular o arquivo frequencies são necessários tanto o gtfs_kit como o gtfs_functions.
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path
from typing import List

import utm
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.geometry as sg
import shapely.ops as so
import pyproj                                                                                                                         
pyproj.__version__                                                                                                                    


from pyproj import CRS                                                                                                                


DIR = Path('/Users/guilhermeiablonovski/OneDrive_UFRGS/_quicko/gtfs_testes')
sys.path.append(str(DIR))

import gtfs_kit as gk
import gtfs_functions as gtfs

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'BA/salvador_ba'

In [ ]:
#Designando o conjunto GTFS a ser lido pelo gtfs-kit
path = DATA_DIR/'salvadorkit.zip'
cairns = (
    gk.read_feed(path, dist_units='km')
    
)
feed = cairns

In [ ]:
#Se o GTFS não possuir arquivo frequencies, gerar um dataframe vazio
d = {'trip_id': [], 'start_time': [], 'end_time':[], 'headway_secs':[], 'exact_times':[] }
feed.frequencies = pd.DataFrame(data=d)

In [ ]:
#Inicializar as variáveis para uso do gtfs_functions
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs(r"/Users/guilhermeiablonovski/OneDrive_UFRGS/_quicko/gtfs_testes/salvadorkit.zip")

In [ ]:
#Usar a função lines_freq para calcular a frequencia de viagens para cada rota

#O cutoff defaul é como o definido na lista a seguir, ou seja a função calcula a frequencia média das viagens nesses intervalos horários
cutoffs = [0,6,9,15.5,19,22,24]
line_freq = gtfs.lines_freq(stop_times, trips, shapes, routes, cutoffs = cutoffs)

In [ ]:
#Realizar um merge para capturar pelo assignar os valores de frequencias das rotas para pelo menos uma de suas viagens associadas em cada janela horária
df = feed.trips
merged = pd.merge(line_freq, df, on='route_id', how='left').drop_duplicates()

In [ ]:
#Filtrar o dataframe para que só contenha dois tipos de combinação de dir_id e direction_id, que são campos redundantes
filtered = merged[((merged.dir_id=='Outbound') & (merged.direction_id==0)) | ((merged.dir_id=='Inbound')&(merged.direction_id==1))]

In [ ]:
#Dropar todas as entradas repetidas para a mesma janela horária para qualquer viagem
final = filtered.drop_duplicates(subset=['trip_headsign','window'])

In [ ]:
#Extrair do dataframe final as variáveis no formato esperado pelo padrão do frequencies
starttime = final['window'].str.split(pat='-', expand=True)
endttime = final['window'].str.split(pat='-', expand=True)
final['start_time']=starttime[0]
final['end_time']=starttime[1]
final['headway_secs']=final['frequency']
final['exact_times']=1

In [ ]:
#Criar uma dataframe com os campos que interessam
novofreq = final[['trip_id','start_time','end_time','headway_secs','exact_times']]

In [ ]:
#Adicionar o dataframe novofreq (gerado com gtfs_functions) ao feed.frequencies (do gtfs-kit) gerado anteriormente ou pré-existente
feed.frequencies = feed.frequencies.append(novofreq)

In [ ]:
#Exportar o resultado
feed.write('/Users/guilhermeiablonovski/OneDrive_UFRGS/_quicko/gtfs_testes/salvadorkit_frequencies.zip')